# meta 분석을 위한 논문 데이터 크롤링

 * ### 현재 진행하는 프로젝트에서 기업문화의 DE&I(Diversity Equity and In)를 위한 메타 분석 진행
 * ### Diversity와 organization을 키워드로 하여 어떤 논문들이 출판되었는지 확인을 위해 데이터 수집
 * ### Google Scholar에서 제공하는 API를 활용

In [ ]:
# !pip3 install -U git+https://github.com/scholarly-python-package/scholarly.git

In [ ]:
# !pip install fake-useragent --upgrade

In [ ]:
# !pip install FreeProxy

In [1]:
# https://pypi.org/project/scholarly/
# scholarlyORG에서 API패키지 제공
from scholarly import scholarly

#출력을 위하여 pprint import
import pprint
pp = pprint.PrettyPrinter(width=100, compact=True)

import pandas as pd

import json

import time

from scholarly import ProxyGenerator

In [2]:
# scholarly 패키지에서 사용할 수 있는 함수 보고 싶은 경우 import 후 help 실행하면 확인 가능
help(scholarly)

Help on _Scholarly in module scholarly._scholarly object:

class _Scholarly(builtins.object)
 |  Class that manages the API for scholarly
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  bibtex(self, object: scholarly.data_types.Publication) -> str
 |      Returns a bibtex entry for a publication that has either Scholar source
 |      or citation source
 |      
 |      :param object: The Publication object for the bibtex exportation
 |      :type object: Publication
 |  
 |  citedby(self, object: scholarly.data_types.Publication) -> scholarly.publication_parser._SearchScholarIterator
 |      Searches Google Scholar for other articles that cite this Publication
 |      and returns a Publication generator.
 |      
 |      :param object: The Publication object for the bibtex exportation
 |      :type object: Publication
 |  
 |  download_mandates_csv(self, filename: str, overwrite: bool = False, includ

In [ ]:
# API페이지에 나와있는 예시 소스코드로 테스트
## 에러 발생! > 에러내용을 살펴보면 __next__에서 에러가 난것으로 검색결과가 없어서 next호출 도중에 문제가 발생한것을 알 수 있음
search_query = scholarly.search_pubs('Perception of physical stability and center of mass of 3D objects')
scholarly.pprint(next(search_query))

In [ ]:
# try catch 구문을 사용하여 처리 가능
## 위 에러메시지에서 출력한 StopIteration 클래스를 except 구문에 넣어 에러 조치
search_query = scholarly.search_pubs('Perception of physical stability and center of mass of 3D objects')
try :
    scholarly.pprint(next(search_query))
except StopIteration:
    print("더이상 논문이 존재하지 않습니다")


In [ ]:
# try-catch구문을 사용하여 diversity와 organization 키워드가 들어있는 논문 검색

try :
    search_query = scholarly.search_pubs('diversity organization')
    pub = next(search_query)

except StopIteration:
    print("더이상 논문이 존재하지 않습니다")


pp.pprint(pub)

In [ ]:
pub['bib']['title']

In [ ]:
type(pub)

In [ ]:
type(pub['bib'])

In [ ]:
pg = ProxyGenerator()
pg.FreeProxies()
scholarly.use_proxy(pg)

In [ ]:
proxy_works = scholarly.use_proxy(pg)

In [ ]:
print(proxy_works)

In [ ]:
# # try-catch구문을 사용하여 diversity와 organization 키워드가 들어있는 논문 검색

# # 전체 정보를 저장하기 위한 데이터 프레임 생성
# df_do = pd.DataFrame()

# # bibiograph정보를 저장하기 위한 데이터 프레인 생성
# df_bib = pd.DataFrame()

# try :
#     # 논문 크롤링 진척률을 확인하기 위해 텍스트 파일 생성
#     file = open('title.txt', 'w')



#     # scholarly 패기지의 search_pubs 함수를 활용하여 diversity organization을 키워드로 논문 검색
#     search_query = scholarly.search_pubs('diversity organization')

#     # 검색 결과가 없을때까지 search_query에 저장된 논문정보 꺼내오기
#     while True :
#         # 다음이 있는경우
#         pub = next(search_query).fill() # dictionary type data
#         # 전체 정보를 저장
#         # 한 행을 데이터 프레임에 입력할 경우 []를 사용하여 행이라는 표시를 해주어야 에러 없이 입력되므로
#         # [pub.items()]로 작성하여 신규 데이터 프레임 생성
#         # 출처 : https://stackoverflow.com/questions/50185926/valueerror-shape-of-passed-values-is-1-6-indices-imply-6-6
#         df = pd.DataFrame([pub.items()], columns=list(pub.keys()))
#         df_do = pd.concat([df_do, df] , axis = 0 )

#         # bibiograph정보를 저장
#         bib = pub['bib']

#         df = pd.DataFrame([bib.items()], columns=list(bib.keys()))
#         df_bib = pd.concat([df_bib, df] , axis = 0 )

#         # 파일에 작업한 논문의 제목 저장
#         file.write(pub['bib']['title'])

# # 더이상 논문이 없는경우 문자열 출력하고 종료
# except StopIteration:
#     print("더이상 논문이 존재하지 않습니다")
#     #논문 크롤링이 끝난 경우 파일 닫기
#     file.close()


# pp.pprint(pub)

In [3]:
def set_new_proxy():
    # while True:
    pg = ProxyGenerator()
    pg.FreeProxies()
    proxy_works = scholarly.use_proxy(pg)

    # if proxy_works:
    #     break
    print("Working proxy:", pg)
    return pg


In [4]:
def get_articleInfo(title):
    while True:
        try:
            search_query = scholarly.search_pubs(title)
            print("Got the results of the query")
            break
        except Exception as e:
            print("Trying new proxy")
            set_new_proxy()

    while True:
        try:
            pub = next(search_query)
            df = pd.DataFrame([pub.items()], columns=list(pub.keys()))
            df_do = pd.concat([df_do, df] , axis = 0 )

            # bibiograph정보를 저장
            bib = pub['bib']

            df = pd.DataFrame([bib.items()], columns=list(bib.keys()))
            df_bib = pd.concat([df_bib, df] , axis = 0 )

            # 파일에 작업한 논문의 제목 저장(바로 제목을 보기 위해 매번 열고닫기)
            file = open('title.txt', 'w')
            file.write(pub['bib']['title'])
            file.close()
        except StopIteration:
            print("더이상 논문이 존재하지 않습니다")
            is_crawling = False
            break
        except Exception as e:
            print("Trying new proxy : ", e)
            set_new_proxy()
    return pub

In [ ]:
# source_data = open("iclr_year_conf_title_author_url_biburl_final_2017-2019.txt")
# line = source_data.readline()
# 전체 정보를 저장하기 위한 데이터 프레임 생성
df_do = pd.DataFrame()

# bibiograph정보를 저장하기 위한 데이터 프레인 생성
df_bib = pd.DataFrame()

is_crawling = True
while is_crawling:
    nowTime = time.strftime("%Y-%m-%d %H:%M:%S")
    print("-------------------------------------------------")
    print(nowTime)
    print(get_articleInfo('diversity organization'))
    print()

-------------------------------------------------
2023-11-05 14:18:49
Trying new proxy
Working proxy: <scholarly._proxy_generator.ProxyGenerator object at 0x1174acb10>


In [ ]:
file.close()

In [ ]:
print(type(pub.keys()))
print(type(pub.items()))

In [ ]:
# You want [data] for pandas to understand they're rows.
# https://stackoverflow.com/questions/50185926/valueerror-shape-of-passed-values-is-1-6-indices-imply-6-6
df = pd.DataFrame([pub.items()], columns=list(pub.keys()))

In [ ]:
df_do = pd.DataFrame()

In [ ]:
pd.concat([df_do, df] , axis = 0 )

In [ ]:
# pd.DataFrame(columns = list(pub.keys()), data = list(pub.values()))

pd.DataFrame(pd.Series(list(pub.values()), index=list(pub.keys()), ignore_index = True))


In [ ]:
list(pub.values())

In [ ]:
pd.DataFrame(pub, columns = list(pub.keys()) )

In [ ]:
pub['author_id']
pub['author_id']

In [ ]:
pp.pprint(scholarly.bibtex(pub))

In [ ]:
from scholarly import scholarly

# Retrieve the author's data, fill-in, and print
# Get an iterator for the author results
search_query = scholarly.search_author('Steven A Cholewiak')
# Retrieve the first result from the iterator
first_author_result = next(search_query)
scholarly.pprint(first_author_result)

# Retrieve all the details for the author
author = scholarly.fill(first_author_result )
scholarly.pprint(author)

# Take a closer look at the first publication
first_publication = author['publications'][0]
first_publication_filled = scholarly.fill(first_publication)
scholarly.pprint(first_publication_filled)

# Print the titles of the author's publications
publication_titles = [pub['bib']['title'] for pub in author['publications']]
print(publication_titles)

# Which papers cited that publication?
citations = [citation['bib']['title'] for citation in scholarly.citedby(first_publication_filled)]
print(citations)

In [ ]:
search_query = scholarly.search_keyword('diversity')
scholarly.pprint(next(search_query))

In [ ]:
search_query = scholarly.search_pubs('diversity')
scholarly.pprint(next(search_query))